## Graph Lattice and Tensile Test examples

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(range(len(nm_orders)),nm_orders)
plt.xlabel("Frame")
plt.ylabel("S2 parameter")

## Show how to manipulate nematic order vector

## Show that bond vectors in opposite directions have the same nematic order